In [3]:
import pandas as pd
import numpy as np
import pprint

In [4]:
# base variables
csv_files = ["ALP", "BS", "DR", "EC", "HD", "JF", "JR", "SS"]
file_suffix = ".csv"

BASE_DIR = "~/Research/wheelchair/data/raw/Max"
OUTPUT_DIR = "~/Research/wheelchair/data/processed/"

# variables for glove types
HYB = "HYB"
PLA = "PLA"

In [5]:
# configuration setup

initials = csv_files[2]
material = PLA
input_file = f"{BASE_DIR}/{material}/{initials}25{material}.csv"
output_file = f"{OUTPUT_DIR}/{material}/{initials}25{material}_per_cycle.csv"

### Helper functions and config

In [19]:
def mean_positive(x):
    positives = x[x > 0]
    if len(x) == 0:
        return np.nan
    return positives.mean()

def mean_negative(x):
    negatives = x[x < 0]
    if len(x) == 0:
        return np.nan
    return negatives.mean()

def peak_negative(x):
    negatives = x[x < 0]
    if len(x) == 0:
        return np.nan
    return negatives.min()

In [20]:
### Build the parameter dictionary
# tangential_force_R[N], radial_force_R[N], axle_force_R[N], moment_z_R[Nm], 'power_z[W]'
xy_force_params = {
    "tangential_force":"N" , 
    "radial_force": "N", 
    "axle_force": "N"
}
z_params = {
    "moment_z_R": "Nm", 
    "power_z": "W"
}

sides = ["L", "R", "avg"]

# build up a dictionary containing the parameters we want to calculate
agg_dict = {}
for col, unit in xy_force_params.items():
    for side in sides:
        col_name = f"{col}_{side}" 
        og_column = f"{col_name}[{unit}]"
        agg_dict[f"{col_name}_pos[{unit}]"] = (og_column, mean_positive)
        agg_dict[f"{col_name}_neg[{unit}]"] = (og_column, mean_negative)
        agg_dict[f"{col_name}_pos_peak[{unit}]"] = (og_column, "max")
        agg_dict[f"{col_name}_neg_peak[{unit}]"] = (og_column, peak_negative)

for col, unit in z_params.items():
    og_column = f"{col}[{unit}]"
    agg_dict[f"{col}_pos[{unit}]"] = (og_column, mean_positive)
    agg_dict[f"{col}_neg[{unit}]"] = (og_column, mean_negative)
    agg_dict[f"{col}_pos_peak[{unit}]"] = (og_column, "max")
    agg_dict[f"{col}_neg_peak[{unit}]"] = (og_column, peak_negative)

### Clean the data
Also add variables

In [21]:
# clean the data
raw_df = pd.read_csv(input_file)

# only use named columns and when the hand cycle is touching
df = raw_df.drop(columns=[c for c in raw_df.columns if "Unnamed" in c])
df = df[df['theta_cop_R[deg]'].notna()]

# add the power[W] calculation for future calculations down the line
df['power_z[W]'] = df['gyro_z_R[rad/s]']*df['moment_z_R[Nm]']

# add the power[W] calculation for future calculations down the line
df['total_force_R[N]'] = np.sqrt(
    df["tangential_force_R[N]"]**2 +
    df["radial_force_R[N]"]**2 +
    df["axle_force_R[N]"]**2
)

df['total_force_L[N]'] =  np.sqrt(
    df["tangential_force_L[N]"]**2 +
    df["radial_force_L[N]"]**2 +
    df["axle_force_L[N]"]**2
)

# Set missing values to 0
# i.e. if they lift their hand up earlier, the force is not recorded, and is set to 0
df['total_force_R[N]'] = df['total_force_R[N]'].fillna(0)
df['total_force_L[N]'] = df['total_force_L[N]'].fillna(0)
df['total_force_average[N]'] = (df['total_force_R[N]'] + df['total_force_L[N]'])/2.0

# Add the average values across the R and L sides too
for param in xy_force_params:
    avg_param_name = f"{param}_avg[N]"
    l_param_name =f"{param}_L[N]"
    r_param_name=f"{param}_R[N]"
    df[avg_param_name]=(df[l_param_name] + df[r_param_name]) / 2.0


In [22]:
# 1. Total force (N)
total_force = df.groupby("cycle[count]")[
    ['total_force_R[N]', 
     'total_force_L[N]',
     'total_force_average[N]']].mean()

In [23]:
# 2-5: split into R&L along with positive, negative values positive and negative values
"""
2. Tangential Force
3. Radial Force
4. Axle Force 
5. Torque
6. Power
"""
avg_kinetics = df.groupby("cycle[count]").agg(**agg_dict)

In [24]:
# 2-5: get the average across the L and R sides

xy_force_params = {
    "tangential_force":"N" , 
    "radial_force": "N", 
    "axle_force": "N"
}

In [25]:
kinetics_df = pd.concat([total_force, avg_kinetics], axis=1)
kinetics_df.describe()

,total_force_R[N],total_force_L[N],total_force_average[N],tangential_force_L_pos[N],tangential_force_L_neg[N],tangential_force_L_pos_peak[N],tangential_force_L_neg_peak[N],tangential_force_R_pos[N],tangential_force_R_neg[N],tangential_force_R_pos_peak[N],...,axle_force_avg_pos_peak[N],axle_force_avg_neg_peak[N],moment_z_R_pos[Nm],moment_z_R_neg[Nm],moment_z_R_pos_peak[Nm],moment_z_R_neg_peak[Nm],power_z_pos[W],power_z_neg[W],power_z_pos_peak[W],power_z_neg_peak[W]
count,25.000000,25.000000,25.000000,25.000000,19.000000,25.000000,19.000000,25.000000,24.000000,25.000000,...,25.000000,0.0,25.000000,24.000000,25.000000,24.000000,24.000000,25.000000,25.000000,25.000000
mean,206.914788,176.523188,191.718988,119.199376,-10.332455,227.368889,-14.733866,119.109885,-12.518549,212.315012,...,133.001040,NaN,20.951046,-5.874539,34.326388,-10.086504,162.284867,-508.199388,270.705619,-846.305512
std,13.538748,17.165909,12.548474,16.636202,6.895572,33.656724,9.904747,16.297345,5.728058,30.003269,...,17.971590,NaN,2.638469,1.639134,3.300059,3.269947,63.043184,114.938980,131.969770,192.507287
min,171.249180,125.557650,164.954070,81.196038,-19.647991,166.221912,-35.002105,91.676512,-24.413029,162.634433,...,99.435838,NaN,17.212435,-8.842556,28.389097,-15.254743,24.842676,-620.760233,-0.434626,-1056.523278
25%,202.216662,168.603289,182.850182,108.949263,-16.068531,201.458196,-21.751037,106.339843,-16.477663,196.135053,...,121.233163,NaN,19.261549,-6.909365,32.489186,-12.522607,131.546924,-584.987671,194.258385,-983.322648
50%,208.843436,179.610493,197.821379,118.642691,-11.249760,229.126988,-13.842296,121.403690,-13.335230,208.872477,...,132.676835,NaN,20.556554,-5.752229,34.026720,-10.740123,167.145252,-533.645193,293.325483,-912.998380
75%,216.491066,187.329488,199.952252,130.223282,-4.012789,248.770433,-7.695081,126.471765,-9.184778,226.958525,...,142.594488,NaN,22.161508,-5.423986,36.247601,-8.700128,202.139319,-484.971273,376.126499,-778.854822
max,224.229807,201.603819,208.469354,145.676767,-0.489214,291.760219,-0.613587,162.386357,-2.035508,290.398457,...,180.598834,NaN,28.357556,-1.895537,42.714286,-2.718319,273.561884,-127.046171,473.782366,-250.397366


In [26]:
kinetics_df

,total_force_R[N],total_force_L[N],total_force_average[N],tangential_force_L_pos[N],tangential_force_L_neg[N],tangential_force_L_pos_peak[N],tangential_force_L_neg_peak[N],tangential_force_R_pos[N],tangential_force_R_neg[N],tangential_force_R_pos_peak[N],...,axle_force_avg_pos_peak[N],axle_force_avg_neg_peak[N],moment_z_R_pos[Nm],moment_z_R_neg[Nm],moment_z_R_pos_peak[Nm],moment_z_R_neg_peak[Nm],power_z_pos[W],power_z_neg[W],power_z_pos_peak[W],power_z_neg_peak[W]
cycle[count],,,,,,,,,,,,,,,,,,,,,
1,211.285229,187.176051,199.230640,145.676767,NaN,198.678537,NaN,162.386357,NaN,211.659111,...,126.419757,NaN,28.357556,NaN,38.609172,NaN,NaN,-127.046171,-0.434626,-250.397366
2,189.143027,168.603289,178.873158,118.642691,NaN,190.627582,NaN,125.896908,-2.334063,224.394466,...,121.233163,NaN,23.788185,-1.895537,38.202178,-2.718319,24.842676,-259.868137,35.657569,-435.680184
3,220.627440,178.574282,199.600861,143.280274,-3.800593,256.950098,-10.008332,133.763409,-2.436396,238.540152,...,180.598834,NaN,24.565915,-2.886644,42.714286,-4.713093,46.698915,-356.898536,76.169051,-616.490040
4,208.843436,190.727025,199.785230,139.384167,-16.266744,248.770433,-35.002105,126.197858,-2.035508,226.958525,...,150.479615,NaN,23.585452,-3.609222,39.744736,-5.275850,66.558552,-404.289680,97.548686,-676.837934
5,215.334888,201.603819,208.469354,143.006403,-2.894206,266.277885,-2.894206,145.497738,-8.378202,227.665199,...,140.339119,NaN,25.025823,-3.632297,37.292722,-5.004520,74.943471,-484.971273,103.460609,-725.327941
6,220.000332,186.755667,203.377999,123.378479,-6.448602,234.061008,-7.956153,129.979548,-10.882072,199.520427,...,138.593348,NaN,22.874905,-5.510847,34.026720,-8.818206,120.766224,-483.806358,194.258385,-720.975518
7,206.575934,164.153062,185.364498,125.960355,-19.276706,273.777241,-28.022583,111.556796,-9.490742,196.135053,...,136.170796,NaN,20.925330,-5.300973,34.619625,-7.858504,123.864678,-471.382017,184.206390,-778.854822
8,211.010450,125.557650,168.284050,106.687998,NaN,229.126988,NaN,119.619250,-5.471767,221.567671,...,132.676835,NaN,21.008661,-5.464991,34.031744,-9.345791,134.107673,-498.246566,229.512204,-809.306779
9,202.517902,195.467630,198.992766,117.566765,-13.842296,199.249807,-13.842296,126.471765,-9.302435,239.892663,...,138.335330,NaN,22.153630,-5.681839,36.247601,-8.345892,145.718678,-553.244125,214.442649,-912.998380


In [13]:
kinetics_df.to_csv(output_file)